#Profiling Pecan Consumers

This notebook profiles the consumers in the Pecan St dataset who have HVAC using a thermal regimes model using whole-home consumption and temperature

##Initializations

In [2]:
rm(list = ls())
options(error = recover)
library('segmented')
library('lubridate')
library('zoo')


Attaching package: ‘zoo’

The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric



In [3]:
setwd("~/EnergyAnalytics/batch/pecan/")
source('define_categories_pecan.r')

setwd("~/EnergyAnalytics/utils/")
source('select_data.r')

setwd('~/EnergyAnalytics/thermal_profiles/profiler/')
source('stateProcessorWrapper.r')
source('stateVisualizerWrapper.r')

setwd("~/EnergyAnalytics/batch/pecan/")

Loading required package: R.oo
Loading required package: R.methodsS3
R.methodsS3 v1.7.0 (2015-02-19) successfully loaded. See ?R.methodsS3 for help.
R.oo v1.19.0 (2015-02-27) successfully loaded. See ?R.oo for help.

Attaching package: ‘R.oo’

The following objects are masked from ‘package:methods’:

    getClasses, getMethods

The following objects are masked from ‘package:base’:

    attach, detach, gc, load, save

R.utils v2.0.0 (2015-02-28) successfully loaded. See ?R.utils for help.

Attaching package: ‘R.utils’

The following object is masked from ‘package:utils’:

    timestamp

The following objects are masked from ‘package:base’:

    cat, commandArgs, getOption, inherits, isOpen, parse, warnings

Warning message:
In removeClass("DataFormatter"): class definition for “DataFormatter” not found (no action taken)Loading required package: lmtest

Attaching package: ‘lmtest’

The following object is masked from ‘package:R.utils’:

    reset

Loading required package: nnet
Loading r

In [4]:
DATA_PATH = '~/S3L_server/energy-data/pecan_street/usage-select/'
DUMP_PATH = '~/S3L_server/energy-data/pecan_street/models_new/'
PLOT_PATH = '~/S3L_server/plots/pecan-street-new/'

In [5]:
# load user names
user_names = read.csv('~/S3L_server/energy-data/pecan_street/metadata/user_names_ids.csv')
user_names$X = NULL

# list already processed files
files.input = list.files(path=DUMP_PATH, pattern = '*_decoded*', full.names = T, recursive = T)
already_done  = lapply(files.input, function(x) {
    tmp = tail(strsplit(x, '/')[[1]],n=2)
    res = tmp[1]
    nfo = strsplit(tmp[2], "_")[[1]]
    uid = nfo[1]; nfo = nfo[2]; 
    return(c(uid,res))
})
already_done = data.frame(do.call('rbind', already_done))
if (length(already_done)>0) names(already_done) <- c("ID", "grain")

# list all data files
files    = list.files(path=DATA_PATH, full.names = T, recursive = T)
files_01 = files[grep('01min',files)]
files_15 = files[grep('15min',files)]
files_60 = files[grep('60min',files)]

# extract ID
users_df = data.frame(UID = as.character(sapply(files_60, function(s) strsplit(tail(strsplit(s, '/')[[1]], 1), '\\.')[[1]][1])))
rownames(users_df) = NULL
# users_df['file_01min'] = files_01
users_df['15min'] = files_15
users_df['60min'] = files_60
    
# build data sources dataframe
users_df = merge(user_names, users_df, by.x="ID", by.y="UID")
users_df = melt(users_df, id=c("ID","name"))
names(users_df)[c(3,4)] = c("grain", "file_orig")  
    
# filter out those IDS already done
users_df = subset(users_df, !((ID %in% already_done$ID & grain %in% already_done$grain)))
print(paste("To process:", dim(users_df)[1], "files"))

[1] "To process: 174 files"


In [6]:
head(users_df)

,ID,name,grain,file_orig
116,3368,Forrest,15min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//15min/3368.csv
165,4505,Milton,15min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//15min/4505.csv
171,4670,Ivan,15min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//15min/4670.csv
283,8121,Alvah,15min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//15min/8121.csv
284,8142,Bertram,15min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//15min/8142.csv
285,8168,Clay,15min,/Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//15min/8168.csv


##Learn thermal regimes models

In [7]:
# function to format data to work with the R thermal profiler code
format_data = function(homeData) {

    # interpolate small gaps
    cat(paste("Initially nrows=",nrow(homeData), '\n'))
    homeData$use = na.spline(homeData$use, maxgap = 12, na.rm=FALSE)
    
    # remove observations with NAs in response
    idx.na = is.na(homeData$use)
    if (sum(idx.na)>0) homeData = homeData[!idx.na,]
    cat(paste("\tRemoved NAs: nrows=",nrow(homeData),"\n"))

    # temperature above reference
    homeData$TemperatureD = homeData$TemperatureF - 65

    # format data as expected by the HMM package
    cur_data = subset(homeData, select = c('date', 'use'))
    names(cur_data)[2] = 'obs'
    cur_data$date = as.character(cur_data$date)
    cur_covar = subset(homeData, select = c('date', 'TemperatureF', 'TemperatureD'))
    cur_covar$date = as.character(cur_covar$date)
    cur_month     = month(cur_data$date)
    cur_covar$TemperatureDWinter = cur_covar$TemperatureD * (cur_month %in% c(0,1,2,3,10,11,12))

    return(list(cur_data, cur_covar))
}


In [8]:
apply_thermal_model = function(cur_data, cur_covar, userName, 
                               dump_path = NULL, 
                               plot_path = NULL) {
  
    # define model learning controls
    controls = list(
        Kmin = 2, Kmax = 6, 
        maxit = 50, 
        nRestarts = 5, 
        tol = 1e-6,
        thresh.R2 = 0.80, 
        thresh.MAPE = 0.20,
        test.periods = 12,
        vis.interval = 3 * 24
    )

    # generate visualization interval; make sure there's data in there
    # TODO: there was an error generated here (indices for subsetting were messed up)
    ok = FALSE
    no.secs    = controls$vis.interval * 3600
    while (!ok) {
        idx_start  = 1
        idx_end    = max(nrow(cur_data)-controls$vis.interval-1, 1)
        start_date = sample(cur_data$date[idx_start:idx_end], 1)
        stop_date  = as.character(as.POSIXct(start_date) + no.secs)
        dat        = subset(cur_data, date >= start_date & date < stop_date)
        if (nrow(na.omit(dat)) > 0) 
          ok = TRUE
    }        

    # learn model
    res = try(stateProcessorWrapper(cur_data, 
                                    cur_covar, 
                                    userName, 
                                    controls = controls,
                                    train.frac = 1.0, 
                                    verbose = F, 
                                    resp.vars = c('(Intercept)', 
                                                  'TemperatureD'), 
                                                  #'TemperatureDWinter'),
                                    dump_path = dump_path))
    if (class(res) == 'try-error') {
        cat(paste('Error in learning model for', userName,'! \n'))
        return(NULL)
    }

    # produce visualizations
    if (!is.null(plot_path)) {
        resv = try(stateVisualizerWrapper(res$decoder, 
                                   res$interpreter, 
                                   plots_path = plot_path, 
                                   interval = c(start_date, stop_date)))
        if (class(resv) == 'try-error') {
            cat('Error in visualizing current user!\n')
        }
    }
    return(NULL)
}


In [9]:
f_wrapper <- function(i) {
    # load data   
    user_id = paste(users_df[i,"ID" ], users_df[i,"name"], sep="_")
    cat(paste('Processing file', users_df[i,'file_orig'], ':', i, '/', nrow(users_df), '\n'))  
    homeData = read.csv(users_df[i,"file_orig"])     
    
    # is there enough data?
    if (is.null(homeData))  {
        cat('Too little data!\n')
        return(NULL)
    }  
    
    # create directory to store models
    dump_path = file.path(DUMP_PATH, paste(user_id, users_df[i,"grain"], sep='/')); 
    dir.create(dump_path, recursive = T)
    # create directory to store plots
    plot_path = file.path(PLOT_PATH, paste(user_id, users_df[i,"name"], sep='/')); 
    dir.create(plot_path, recursive = T)

    # format dataset
    res = format_data(homeData); cur_data = res[[1]]; cur_covar = res[[2]];
    
    # apply model to data
    res = apply_thermal_model(cur_data, cur_covar, user_id, 
                            dump_path = dump_path, 
                            plot_path = plot_path)  
    return(NULL)
}


In [10]:
# library('parallel')
# res = mclapply(1:nrow(users_df), mc.cores = 3, f_wrapper)


In [11]:
setwd("~/EnergyAnalytics/batch/pecan/")
source('define_categories_pecan.r')

setwd("~/EnergyAnalytics/utils/")
source('select_data.r')
    
setwd('~/EnergyAnalytics/thermal_profiles/profiler/')
source('stateProcessorWrapper.r')
source('stateVisualizerWrapper.r')

setwd("~/EnergyAnalytics/batch/pecan/")
#fit = f_wrapper(474)

In [ ]:
which(users_df$name=='Lucius' & users_df$grain=='60min')

In [ ]:
users_df = users_df[with(users_df, order(ID,grain,name)),]
for (i in 1:nrow(users_df)) {
    cat(paste("*****", i, ":", users_df[i,c("name", "ID")], "*****"))
    f_wrapper(i)
}

***** 1 : 198 ***** ***** 1 : 3368 *****Processing file /Users/adrianalbert/S3L_server/energy-data/pecan_street/usage-select//15min/3368.csv : 1 / 174 


Warning message:
In dir.create(dump_path, recursive = T): '/Users/adrianalbert/S3L_server/energy-data/pecan_street/models_new//3368_Forrest/15min' already existsWarning message:
In dir.create(plot_path, recursive = T): '/Users/adrianalbert/S3L_server/plots/pecan-street-new//3368_Forrest/Forrest' already exists

Initially nrows= 18908 
	Removed NAs: nrows= 17852 
*** Initializing DataFormatter (3368_Forrest) ***
   user  system elapsed 
  0.001   0.000   0.001 
*** Adding covariates data for DataFormatter3368_Forrest ***
   user  system elapsed 
  0.009   0.000   0.010 
*** Initializing StateDecoder (3368_Forrest) ***
   user  system elapsed 
  0.007   0.000   0.008 
***** HMM Cross-Validation K = 2 ******
iteration 0 logLik: -9662.849 
Bad HMM fit; re-estimating...
iteration 0 logLik: -9729.372 
Bad HMM fit; re-estimating...
iteration 0 logLik: -9742.904 
Bad HMM fit; re-estimating...
iteration 0 logLik: -9429.142 
Bad HMM fit; re-estimating...
iteration 0 logLik: -9839.547 
iteration 5 logLik: 956.3785 
Bad HMM fit; re-estimating...
iteration 0 logLik: -9827.108 
Bad HMM fit; re-estimating...
[1] "MAPE=NA; R2=NA"
***** HMM Cross-Validation K = 3 ******
iteration 0 logLik: -9409.459 
iteration 5 logLik: 2186.632 
Bad HMM fit; re-estimating...
iteration 0 logLik: -9513.42 
iteration 5 logLik: 

Warning message:
In dir.create(dump_path, recursive = T): '/Users/adrianalbert/S3L_server/energy-data/pecan_street/models_new//3368_Forrest/60min' already existsWarning message:
In dir.create(plot_path, recursive = T): '/Users/adrianalbert/S3L_server/plots/pecan-street-new//3368_Forrest/Forrest' already exists

Initially nrows= 4727 
	Removed NAs: nrows= 4463 
*** Initializing DataFormatter (3368_Forrest) ***
   user  system elapsed 
      0       0       0 
*** Adding covariates data for DataFormatter3368_Forrest ***
   user  system elapsed 
  0.001   0.000   0.001 
*** Initializing StateDecoder (3368_Forrest) ***
   user  system elapsed 
  0.006   0.000   0.006 
***** HMM Cross-Validation K = 2 ******
iteration 0 logLik: -2024.168 
iteration 5 logLik: 355.2221 
Bad HMM fit; re-estimating...
iteration 0 logLik: -2075.433 
iteration 5 logLik: 248.0721 
Bad HMM fit; re-estimating...
iteration 0 logLik: -1881.181 
iteration 5 logLik: 528.4397 
Bad HMM fit; re-estimating...
iteration 0 logLik: -2100.617 
iteration 5 logLik: 133.7824 
Bad HMM fit; re-estimating...
iteration 0 logLik: -1984.077 
iteration 5 logLik: 401.1595 
Bad HMM fit; re-estimating...
iteration 0 logLik: -2094.575 
iteration 5 logLik: 182.0096 
Bad HMM fit; re-estimating...
[1] "MAPE=NA; R2=NA"
***** HMM Cross-Validation K = 3 

...

Warning message:
In dir.create(plot_path, recursive = T): '/Users/adrianalbert/S3L_server/plots/pecan-street-new//8197_Manuel/Manuel' already exists

Initially nrows= 14938 
	Removed NAs: nrows= 14924 
*** Initializing DataFormatter (8197_Manuel) ***
   user  system elapsed 
  0.002   0.000   0.003 
*** Adding covariates data for DataFormatter8197_Manuel ***
   user  system elapsed 
  0.004   0.000   0.005 
*** Initializing StateDecoder (8197_Manuel) ***
   user  system elapsed 
  0.005   0.000   0.005 
***** HMM Cross-Validation K = 2 ******
iteration 0 logLik: -14127.86 
iteration 5 logLik: -12902.75 
iteration 10 logLik: -11880.63 
iteration 15 logLik: -11832.04 
iteration 20 logLik: -11829.61 
converged at iteration 24 with logLik: -11829.5 
[1] "MAPE=0.2224386998246; R2=0.753225085753035"
***** HMM Cross-Validation K = 3 ******
iteration 0 logLik: -14125.14 
iteration 5 logLik: -12667.27 
iteration 10 logLik: -11754.67 
iteration 15 logLik: -11253.91 
iteration 20 logLik: -11154.88 
iteration 25 logLik: -11141.82 
iteration 30 logLik: -11133.9 
iteration 35 logLik: -11128.35 
iteration 40 logLik: -11124.66 
iteration 45 logLik:

Warning message:
In dir.create(file.path(plots_path, visualizer@UID)): '/Users/adrianalbert/S3L_server/plots/pecan-street-new//8197_Manuel/Manuel/8197_Manuel' already existsWarning message:
In matrix(as.numeric(myMat), ncol = 24, byrow = T): data length [14924] is not a sub-multiple or multiple of the number of rows [622]

      obs             State        Index                 Time   
 Min.   :1.339   C.Lo (1):48   Min.   : 1   Mon,08/12 00:00: 1  
 1st Qu.:3.180   N (2)   :10   1st Qu.:19   Mon,08/12 01:00: 1  
 Median :5.206   N (3)   :15   Median :37   Mon,08/12 02:00: 1  
 Mean   :5.261                 Mean   :37   Mon,08/12 03:00: 1  
 3rd Qu.:7.397                 3rd Qu.:55   Mon,08/12 04:00: 1  
 Max.   :9.499                 Max.   :73   Mon,08/12 05:00: 1  
                                            (Other)        :67  
  TemperatureD    
 Min.   :0.01199  
 1st Qu.:0.02383  
 Median :0.05535  
 Mean   :0.04212  
 3rd Qu.:0.05535  
 Max.   :0.05535  
                  


Warning message:
: The shape palette can deal with a maximum of 6 discrete values because
more than 6 becomes difficult to discriminate; you have 7. Consider
specifying shapes manually. if you must have them.Warning message:
: The shape palette can deal with a maximum of 6 discrete values because
more than 6 becomes difficult to discriminate; you have 7. Consider
specifying shapes manually. if you must have them.Warning message:
In loop_apply(n, do.ply): Removed 48 rows containing missing values (geom_point).Warning message:
In loop_apply(n, do.ply): Removed 48 rows containing missing values (geom_point).Warning message:
: The shape palette can deal with a maximum of 6 discrete values because
more than 6 becomes difficult to discriminate; you have 7. Consider
specifying shapes manually. if you must have them.